# ARC - Crop Monitoring with CDSE Sentinel-2 Data

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/profLewis/ARC/blob/main/notebooks/test_cdse.ipynb)

This notebook demonstrates the ARC (Archetypes for Crop Monitoring) pipeline using
Sentinel-2 L2A data from the **Copernicus Data Space Ecosystem (CDSE)**.

## Prerequisites

You need CDSE credentials. Two authentication methods are supported:

### Option 1: S3 Access Keys (recommended)
1. Register at https://dataspace.copernicus.eu
2. Go to https://eodata.dataspace.copernicus.eu (S3 credentials dashboard)
3. Generate an S3 access key and secret key
4. Enter them when prompted below

### Option 2: Username / Password
1. Register at https://dataspace.copernicus.eu
2. Use your login email and password when prompted below

## 1. Install ARC

In [ ]:
!pip install -q https://github.com/profLewis/ARC/archive/refs/heads/main.zip

## 2. Set up CDSE credentials

Choose your authentication method and enter credentials.

In [ ]:
import os
import getpass

print("CDSE Authentication")
print("="*40)
print("1: S3 Access Keys (recommended)")
print("2: Username / Password")
auth_method = input("Choose method [1/2]: ").strip()

if auth_method == "1":
    os.environ["CDSE_S3_ACCESS_KEY"] = getpass.getpass("CDSE S3 Access Key: ")
    os.environ["CDSE_S3_SECRET_KEY"] = getpass.getpass("CDSE S3 Secret Key: ")
    print("S3 credentials set.")
else:
    os.environ["CDSE_USERNAME"] = input("CDSE Username (email): ")
    os.environ["CDSE_PASSWORD"] = getpass.getpass("CDSE Password: ")
    print("Username/password credentials set.")

In [ ]:
from pystac_client import Client

print("Testing CDSE credentials...")

# Test 1: STAC API is reachable (no auth needed)
catalog = Client.open("https://stac.dataspace.copernicus.eu/v1")
print(f"  STAC API: OK ({catalog.title})")

# Test 2: Check which credentials are configured
has_s3 = os.environ.get("CDSE_S3_ACCESS_KEY") and os.environ.get("CDSE_S3_SECRET_KEY")
has_pw = os.environ.get("CDSE_USERNAME") and os.environ.get("CDSE_PASSWORD")

if has_s3:
    # Test S3 access by reading a small file from a known Sentinel-2 item
    from osgeo import gdal
    gdal.SetConfigOption("AWS_S3_ENDPOINT", "eodata.dataspace.copernicus.eu")
    gdal.SetConfigOption("AWS_ACCESS_KEY_ID", os.environ["CDSE_S3_ACCESS_KEY"])
    gdal.SetConfigOption("AWS_SECRET_ACCESS_KEY", os.environ["CDSE_S3_SECRET_KEY"])
    gdal.SetConfigOption("AWS_VIRTUAL_HOSTING", "FALSE")
    gdal.SetConfigOption("GDAL_DISABLE_READDIR_ON_OPEN", "EMPTY_DIR")

    # Search for one recent item to test access
    search = catalog.search(collections=["sentinel-2-l2a"], max_items=1)
    items = list(search.items())
    if items:
        item = items[0]
        # Try to open the B02 band
        href = item.assets["B02_10m"].href
        s3_path = "/" + href[5:] if href.startswith("s3://") else href
        ds = gdal.Open(f"/vsis3{s3_path}")
        if ds is not None:
            print(f"  S3 access:  OK (read {item.id} B02)")
            ds = None
        else:
            print("  S3 access:  FAILED - check your S3 access key and secret key")
    else:
        print("  S3 access:  could not find test item")

elif has_pw:
    import requests
    token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    resp = requests.post(token_url, data={
        "grant_type": "password",
        "client_id": "cdse-public",
        "username": os.environ["CDSE_USERNAME"],
        "password": os.environ["CDSE_PASSWORD"],
    })
    if resp.status_code == 200 and "access_token" in resp.json():
        print("  OAuth2 token: OK")
    else:
        print(f"  OAuth2 token: FAILED (HTTP {resp.status_code}) - check username/password")
else:
    print("  WARNING: No credentials found. Set S3 keys or username/password above.")

print("\nCredential test complete.")

## 3. Test archetype generation (no credentials needed)

This tests the forward model and ensemble generation — no satellite data is downloaded.

In [ ]:
import arc
import numpy as np
import matplotlib.pyplot as plt

doys = np.arange(1, 366, 5)
angs = (
    np.array([30] * len(doys)),   # VZA
    np.array([10] * len(doys)),   # SZA
    np.array([120] * len(doys)),  # RAA
)

s2_refs, pheo_samples, bio_samples, orig_bios, soil_samples = arc.generate_arc_refs(
    doys=doys,
    start_of_season=150,
    growth_season_length=45,
    num_samples=10000,
    angs=angs,
    crop_type='maize'
)

print(f"Archetype spectra shape: {s2_refs.shape}  (10 bands x {s2_refs.shape[1]} dates x {s2_refs.shape[2]} samples)")
print(f"Biophysical parameters shape: {orig_bios.shape}  (7 params x dates x samples)")

# Plot max NDVI vs max LAI
max_lai = np.nanmax(orig_bios[4], axis=0)
ndvi = (s2_refs[7] - s2_refs[3]) / (s2_refs[7] + s2_refs[3])
max_ndvi = np.nanmax(ndvi, axis=0)

plt.figure(figsize=(8, 6))
plt.plot(max_ndvi, max_lai / 100, 'o', ms=3, alpha=0.05)
plt.xlabel('Max NDVI')
plt.ylabel('Max LAI (m$^2$/m$^2$)')
plt.title('Archetype ensemble: NDVI vs LAI')
plt.grid(True, alpha=0.3)
plt.show()
print("Archetype generation: OK")

## 4. Test CDSE Sentinel-2 data retrieval

Download S2 data for a South African wheat field from the CDSE STAC API.

In [ ]:
import eof
from pathlib import Path

arc_dir = os.path.dirname(os.path.realpath(arc.__file__))
geojson_path = f"{arc_dir}/test_data/SF_field.geojson"

S2_data_folder = Path.home() / "Downloads/SF_field_cdse"
S2_data_folder.mkdir(parents=True, exist_ok=True)

print("Retrieving Sentinel-2 data from CDSE...")
s2_refs, s2_uncs, s2_angles, doys, mask, geotransform, crs = eof.get_s2_official_data(
    start_date="2022-07-15",
    end_date="2022-11-30",
    geojson_path=geojson_path,
    S2_data_folder=str(S2_data_folder),
    source='cdse',
)

print(f"\nResults:")
print(f"  s2_refs shape:   {s2_refs.shape}  (images, bands, H, W)")
print(f"  s2_uncs shape:   {s2_uncs.shape}")
print(f"  s2_angles shape: {s2_angles.shape}  [SZA, VZA, RAA]")
print(f"  doys:            {doys}")
print(f"  mask shape:      {mask.shape}")
print(f"  geotransform:    {geotransform}")
print(f"  N images:        {s2_refs.shape[0]}")
print(f"  Valid pixels:    {(~mask).sum()} / {mask.size}")
print(f"  Reflectance range: [{np.nanmin(s2_refs):.4f}, {np.nanmax(s2_refs):.4f}]")

In [ ]:
# Plot NDVI time series from CDSE data
ndvi = (s2_refs[:, 7] - s2_refs[:, 2]) / (s2_refs[:, 7] + s2_refs[:, 2])
mean_ndvi = np.nanmean(ndvi, axis=(1, 2))

plt.figure(figsize=(12, 4))
valid = np.isfinite(mean_ndvi)
plt.plot(doys[valid], mean_ndvi[valid], '--o', label='Mean NDVI (CDSE)')
plt.xlabel('Day of year')
plt.ylabel('NDVI')
plt.title('CDSE Sentinel-2: Mean NDVI Time Series')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Run the full ARC pipeline with CDSE

This runs the complete pipeline: CDSE data retrieval, archetype generation,
KNN search, and data assimilation.

In [ ]:
START_OF_SEASON = 225
CROP_TYPE = 'wheat'
NUM_SAMPLES = 100000
GROWTH_SEASON_LENGTH = 45

print("Running full ARC pipeline with CDSE data source...")
scale_data, post_bio_tensor, post_bio_unc_tensor, mask, doys = arc.arc_field(
    s2_start_date="2022-07-15",
    s2_end_date="2022-11-30",
    geojson_path=geojson_path,
    start_of_season=START_OF_SEASON,
    crop_type=CROP_TYPE,
    output_file_path=f'{S2_data_folder}/SF_field_cdse.npz',
    num_samples=NUM_SAMPLES,
    growth_season_length=GROWTH_SEASON_LENGTH,
    S2_data_folder=str(S2_data_folder),
    data_source='cdse',
)

print(f"\nPipeline complete!")
print(f"  post_bio_tensor shape: {post_bio_tensor.shape}  (pixels, 7 params, dates)")
print(f"  post_bio_unc_tensor shape: {post_bio_unc_tensor.shape}")

## 6. Plot LAI time series

In [ ]:
plt.figure(figsize=(12, 6))
step = max(1, post_bio_tensor.shape[0] // 100)
plt.plot(doys, post_bio_tensor[::step, 4].T / 100, '-', lw=1.5, alpha=0.6)
plt.ylabel('LAI (m$^2$/m$^2$)')
plt.xlabel('Day of year')
plt.title('LAI time series (CDSE data source)')
plt.grid(True, alpha=0.3)
plt.show()

## 7. Plot LAI maps

In [ ]:
lai = post_bio_tensor[:, 4].T / 100  # (dates, pixels)
nrows = int(np.ceil(len(doys) / 5))
fig, axs = plt.subplots(ncols=5, nrows=nrows, figsize=(20, 4 * nrows))
axs = axs.ravel()

for i in range(len(doys)):
    lai_map = np.full(mask.shape, np.nan)
    lai_map[~mask] = lai[i]
    im = axs[i].imshow(lai_map, vmin=0, vmax=7)
    fig.colorbar(im, ax=axs[i], shrink=0.8, label='LAI (m$^2$/m$^2$)')
    axs[i].set_title(f'DOY: {doys[i]}')

for i in range(len(doys), len(axs)):
    axs[i].axis('off')

plt.suptitle('LAI Maps (CDSE data source)', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

## 8. Validate output shapes and values

In [ ]:
print("Validation checks:")
print(f"  post_bio_tensor ndim == 3: {post_bio_tensor.ndim == 3}")
print(f"  7 biophysical params: {post_bio_tensor.shape[1] == 7}")
print(f"  N dates matches doys: {post_bio_tensor.shape[2] == len(doys)}")

lai_phys = post_bio_tensor[:, 4, :] / 100
print(f"  LAI range: [{lai_phys.min():.2f}, {lai_phys.max():.2f}] m2/m2")
print(f"  LAI range valid (0-10): {lai_phys.min() >= 0 and lai_phys.max() <= 10}")

cab_phys = post_bio_tensor[:, 1, :] / 100
print(f"  Cab range: [{cab_phys.min():.2f}, {cab_phys.max():.2f}] ug/cm2")

print("\nAll checks passed!" if post_bio_tensor.shape[1] == 7 else "\nSome checks failed.")